In [14]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Load and prepare data
df = pd.read_csv("nba_22features.csv")
df['Game Date'] = pd.to_datetime(df['Game Date'])

# Step 2: Filter 2016 playoffs for testing
playoffs_2016 = df[(df['Game Date'].dt.year == 2016) & (df['is_playoffs'] == True)]
train_df = df[~df.index.isin(playoffs_2016.index)]

print(playoffs_2016.columns)
print(train_df.columns)

Index(['game_id', 'Game Date', 'Home Days Since Last Game',
       'Home Games Last 7 Days', 'Home Is Back-to-Back',
       'Away Days Since Last Game', 'Away Games Last 7 Days',
       'Away Is Back-to-Back', 'Away Team Recent 10 Win Rate',
       'Home Team Recent 10 Win Rate', 'Home Team Recent 3 Months Win Rate',
       'Away Team Recent 3 Months Win Rate', 'home_pts_avg_last10',
       'home_reb_avg_last10', 'home_ast_avg_last10',
       'home_plus_minus_avg_last10', 'away_pts_avg_last10',
       'away_reb_avg_last10', 'away_ast_avg_last10',
       'away_plus_minus_avg_last10', 'target', 'home_injury_count',
       'away_injury_count', 'is_playoffs', 'is_playoffs_game_df'],
      dtype='object')
Index(['game_id', 'Game Date', 'Home Days Since Last Game',
       'Home Games Last 7 Days', 'Home Is Back-to-Back',
       'Away Days Since Last Game', 'Away Games Last 7 Days',
       'Away Is Back-to-Back', 'Away Team Recent 10 Win Rate',
       'Home Team Recent 10 Win Rate', 'Home Tea

In [15]:
from sklearn.discriminant_analysis import StandardScaler


X_train = train_df.drop(columns=['game_id', 'Game Date', 'is_playoffs', 'target'])
y_train = train_df['target']

X_test = playoffs_2016.drop(columns=['game_id', 'Game Date', 'is_playoffs', 'target'])
y_test = playoffs_2016['target']


X_train_clean = X_train.dropna()
y_train_clean = y_train[X_train_clean.index]


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_clean)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(max_iter=3000)
model.fit(X_train_clean, y_train_clean)

y_pred = model.predict(X_test)

print("Accuracy on 2016 Playoffs:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))



Accuracy on 2016 Playoffs: 0.7093023255813954
              precision    recall  f1-score   support

         0.0       0.59      0.36      0.44        28
         1.0       0.74      0.88      0.80        58

    accuracy                           0.71        86
   macro avg       0.66      0.62      0.62        86
weighted avg       0.69      0.71      0.69        86



In [16]:
playoffs_2016 = playoffs_2016.copy()
playoffs_2016['prediction'] = y_pred
playoffs_2016['correct'] = (y_pred == y_test)


finals_games = playoffs_2016[
    (playoffs_2016['Game Date'] >= '2016-06-02') &
    (playoffs_2016['Game Date'] <= '2016-06-19')
]

print("Model Predictions on 2016 NBA Finals:")
print(finals_games[['Game Date', 'game_id', 'target', 'prediction', 'correct']])

finals_accuracy = accuracy_score(finals_games['target'], finals_games['prediction'])
print(f"\nAccuracy on NBA Finals only: {finals_accuracy:.2%}")

Model Predictions on 2016 NBA Finals:
       Game Date   game_id  target  prediction  correct
19962 2016-06-02  41500401     1.0         0.0    False
19963 2016-06-05  41500402     1.0         1.0     True
19964 2016-06-08  41500403     1.0         1.0     True
19965 2016-06-10  41500404     0.0         1.0    False
19966 2016-06-13  41500405     0.0         1.0    False
19967 2016-06-16  41500406     1.0         1.0     True
19968 2016-06-19  41500407     0.0         1.0    False

Accuracy on NBA Finals only: 42.86%


In [17]:
games_df = pd.read_csv("game.csv")

# Step 2: Extract relevant columns
game_meta = games_df[['game_id', 'team_name_home', 'team_name_away']]

# Step 3: Merge with your playoff predictions
finals_with_teams = finals_games.merge(game_meta, on='game_id', how='left')

# Step 4: Create a readable summary
def summarize(row):
    actual = "Home win" if row['target'] == 1 else "Away win"
    predicted = "Home" if row['prediction'] == 1 else "Away"
    status = "✅" if row['correct'] else "❌"
    return f"{row['team_name_home']} vs {row['team_name_away']} | Actual: {actual}, Predicted: {predicted} {status}"

finals_with_teams['summary'] = finals_with_teams.apply(summarize, axis=1)

# Step 5: Show final readable report
print("🏀 2016 NBA Finals Model Predictions:\n")
print(finals_with_teams[['Game Date', 'summary']])

🏀 2016 NBA Finals Model Predictions:

   Game Date                                            summary
0 2016-06-02  Golden State Warriors vs Cleveland Cavaliers |...
1 2016-06-05  Golden State Warriors vs Cleveland Cavaliers |...
2 2016-06-08  Cleveland Cavaliers vs Golden State Warriors |...
3 2016-06-10  Cleveland Cavaliers vs Golden State Warriors |...
4 2016-06-13  Golden State Warriors vs Cleveland Cavaliers |...
5 2016-06-16  Cleveland Cavaliers vs Golden State Warriors |...
6 2016-06-19  Golden State Warriors vs Cleveland Cavaliers |...


In [18]:
def describe_game(row):
    actual = "Home win" if row['target'] == 1 else "Away win"
    predicted = "Home" if row['prediction'] == 1 else "Away"
    correct = "✅" if row['correct'] else "❌"
    return f"{actual} / Predicted: {predicted} {correct}"

# Apply description
finals_with_teams['result_summary'] = finals_with_teams.apply(describe_game, axis=1)

# Final summary table
final_summary = finals_with_teams[[
    'Game Date', 'team_name_home', 'team_name_away', 'result_summary'
]]

# Sort by date (just in case)
final_summary = final_summary.sort_values(by='Game Date')

# Display
print("📊 Combined Finals Predictions:\n")
print(final_summary)

📊 Combined Finals Predictions:

   Game Date         team_name_home         team_name_away  \
0 2016-06-02  Golden State Warriors    Cleveland Cavaliers   
1 2016-06-05  Golden State Warriors    Cleveland Cavaliers   
2 2016-06-08    Cleveland Cavaliers  Golden State Warriors   
3 2016-06-10    Cleveland Cavaliers  Golden State Warriors   
4 2016-06-13  Golden State Warriors    Cleveland Cavaliers   
5 2016-06-16    Cleveland Cavaliers  Golden State Warriors   
6 2016-06-19  Golden State Warriors    Cleveland Cavaliers   

                 result_summary  
0  Home win / Predicted: Away ❌  
1  Home win / Predicted: Home ✅  
2  Home win / Predicted: Home ✅  
3  Away win / Predicted: Home ❌  
4  Away win / Predicted: Home ❌  
5  Home win / Predicted: Home ✅  
6  Away win / Predicted: Home ❌  
